# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# import data 
transactions = pd.read_csv("../data/bank_transactions.csv")

transactions.sample(5)

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
914171,CASH_IN,93845.30,C1978097924,104016.0,197861.3,C1867397089,2287.57,0.00,0,0
703592,PAYMENT,13292.30,C7223443,0.0,0.0,M1317509488,0.00,0.00,0,0
650676,PAYMENT,16673.31,C1211515696,0.0,0.0,M1412898547,0.00,0.00,0,0
913384,CASH_OUT,102731.02,C757903603,0.0,0.0,C743920731,2172237.91,2274968.93,0,0
234297,CASH_OUT,166093.34,C1681357071,0.0,0.0,C1854124318,318997.14,485090.48,0,0


In [4]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   type            1000000 non-null  object 
 1   amount          1000000 non-null  float64
 2   nameOrig        1000000 non-null  object 
 3   oldbalanceOrg   1000000 non-null  float64
 4   newbalanceOrig  1000000 non-null  float64
 5   nameDest        1000000 non-null  object 
 6   oldbalanceDest  1000000 non-null  float64
 7   newbalanceDest  1000000 non-null  float64
 8   isFraud         1000000 non-null  int64  
 9   isFlaggedFraud  1000000 non-null  int64  
dtypes: float64(5), int64(2), object(3)
memory usage: 76.3+ MB


## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

There are no missing values but there are non predictive columns including nameOrig and nameDest and isFraud flagged. I will drop these columns from the data set. 

In [5]:
transactions.drop(columns=["nameOrig", "nameDest", "isFlaggedFraud"], inplace=True)


In [6]:
transactions.sample(5)

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
930505,CASH_IN,339236.01,8780153.00,9119389.01,379296.17,40060.16,0
822326,CASH_OUT,152353.69,0.00,0.00,623768.73,776122.42,0
541711,PAYMENT,2818.25,0.00,0.00,0.00,0.00,0
329285,PAYMENT,14386.72,0.00,0.00,0.00,0.00,0
899921,PAYMENT,6970.02,357968.51,350998.49,0.00,0.00,0


## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

Fraudulent transactions are usually transaction type Transfer or Cash out. I can use one-hot encoding to transform the type column from categorical to binary format. 

In [14]:
encoded_df = pd.get_dummies(transactions, columns=['type'])

encoded_df.head(5)

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,983.09,36730.24,35747.15,0.00,0.00,0,False,False,False,True,False
1,55215.25,99414.00,44198.75,0.00,0.00,0,False,False,False,True,False
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,True,False,False,False,False
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0,False,False,False,False,True
4,67990.14,0.00,0.00,625317.04,693307.19,0,False,True,False,False,False


## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

Yes, there are many more non-fradulent transactions than fradulent ones. This might make the model biased to predicting non-fradulent transactions and not predict the fradulent ones well. I can use SMOTE to generate more samples of non-fradulent transactions. However this will come after splitting the data for fitting the model and not in the preprocessing step.


## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.



In [15]:
# write out newly transformed dataset to your folder
encoded_df.to_csv('../data/bank_transactions_cleaned.csv', index=False)